# Aggregators

Using `Recorder` to see the recipes.

In [1]:
import graphblas as gb
from graphblas import Recorder, Matrix, Vector, agg

In [2]:
# We can create a recorder and have it be alive until we call `rec.stop()`.
# One can do `rec.clear()` to clear the data.
rec = Recorder()
A = Matrix.from_coo(
    [3, 0, 3, 5, 6, 0, 6, 1, 6, 2, 5],
    [0, 1, 2, 2, 2, 3, 3, 4, 4, 5, 5],
    [3, 2, 3, 1, 5, 3, 7, 8, 3, 1, 7],
    dtype="int32",
    name="A",
)
A

"A"        nvals  nrows  ncols  dtype   format
gb.Matrix     11      7      6  INT32  bitmapr
----------------------------------------------
   0  1  2  3  4  5
0     2     3      
1              8   
2                 1
3  3     3         
4                  
5        1        7
6        5  7  3

In [3]:
rec

gb.Recorder


In [4]:
# You can also record within a context.
# Only one recorder records at a given time, so `rec` does not save these calls
with Recorder() as rec2:
    result = A.reduce_rowwise(agg.min).new()
result

"v_0"      nvals  size  dtype  format
gb.Vector      6     7  INT32  bitmap
-------------------------------------
index  0  1  2  3 4  5  6
value  2  8  1  3    1  3

In [5]:
# Here we see our recipe simply used a monoid as one would expect for MIN
rec2

gb.Recorder


In [6]:
rec.clear()
A.reduce_columnwise(agg.count).new()

"v_1"      nvals  size  dtype  format
gb.Vector      6     6  INT32    full
-------------------------------------
index  0  1  2  3  4  5
value  1  1  3  2  2  2

### Look at recipes for all the aggregators

In [7]:
from IPython.display import display, HTML, Markdown

In [8]:
skipped = set()
for aggname in sorted(dir(agg)):
    aggregator = getattr(agg, aggname)
    if not isinstance(aggregator, agg.Aggregator):
        continue
    try:
        # The type-handing of aggregators still needs work, so hack it for now
        if aggname in {"all", "any"}:
            B = A.dup(dtype=bool)
        elif aggname in {"bitwise_all", "bitwise_any"}:
            B = A.dup(dtype="UINT64")
        else:
            B = A
        # result = B.reduce_columnwise(aggregator).new()
        # result = B.reduce_scalar(aggregator).new()
        # result = B.T.reduce_rowwise(aggregator).new()
        # result = B.T.reduce_columnwise(aggregator).new()
        # result = B.T.reduce_scalar(aggregator).new()
        # display(Markdown(r._repr_html_()))
        display(Markdown(f"# {aggname}"))
        with Recorder(max_rows=None) as r:
            result = B.reduce_rowwise(aggregator).new(name="rowwise")
            display(HTML(result._repr_html_()))
            display(r)
        with Recorder(max_rows=None) as r:
            result = B.reduce_scalar(aggregator).new(name="scalar")
            display(HTML(result._repr_html_()))
            display(r)
    except Exception:
        skipped.add(aggname)

# L0norm

gb.Recorder


gb.Recorder


# L1norm

gb.Recorder


...

gb.Recorder


gb.Recorder


# vars

gb.Recorder


gb.Recorder


In [9]:
skipped

{'argmax', 'argmin', 'first_index', 'last_index'}

In [10]:
try:
    A.reduce_scalar(agg.argmin).new()
except ValueError as e:
    exc = e
exc

ValueError('Aggregator argmin may not be used with Matrix.reduce_scalar.')

In [11]:
v = Vector.from_coo([1, 3, 4, 6], [1, 1, 2, 0])
v

"v_189"    nvals  size  dtype  format
gb.Vector      4     7  INT64  bitmap
-------------------------------------
index 0  1 2  3  4 5  6
value    1    1  2    0

In [12]:
skipped = set()
for aggname in sorted(dir(agg)):
    aggregator = getattr(agg, aggname)
    if not isinstance(aggregator, agg.Aggregator):
        continue
    try:
        # The type-handing of aggregators still needs work, so hack it for now
        if aggname in {"all", "any"}:
            w = v.dup(dtype=bool)
        elif aggname in {"bitwise_all", "bitwise_any"}:
            w = v.dup(dtype="UINT64")
        else:
            w = v
        display(Markdown(f"# {aggname}"))
        with Recorder(max_rows=None) as r:
            result = w.reduce(aggregator).new()
            display(HTML(result._repr_html_()))
            display(r)
    except Exception:
        skipped.add(aggname)

# L0norm

gb.Recorder


# L1norm

gb.Recorder


# L2norm

...

gb.Recorder


# varp

gb.Recorder


# vars

gb.Recorder


In [13]:
skipped

set()